In [108]:
import os
import bs4
from bs4 import BeautifulSoup as soup
import pandas as pd
import spacy
import re
import en_core_web_sm
import keras
import tensorflow
from pathlib import Path
from pathlib import PureWindowsPath
import helper_parsing_functions
from helper_parsing_functions import *
import csv
from dateparser.search import search_dates
import arrow
import datetime

nlp = en_core_web_sm.load()

def extract_name_pair(name, input_text):
    out_list = []
    if not (input_text==[]):
        #         s_out = extract_chunks(input_text)  
        sent =  " ".join(str(x) for x in input_text)
        #     doc = nlp(input_text)
        doc = nlp(sent)        
        #         doc = nlp(input_text)
        spans = list(doc.ents)  #+ list(doc.noun_chunks)

        for item in spans:
            try:
                item = re.sub(pattern='[^a-zA-Z0-9 -]',repl='',string=item.string)            
                if (re.search(item,name,flags=re.IGNORECASE)):
                    out_list.append(item)

            except:
                pass
            
    return out_list

In [3]:
collection = "html"

# n_test=50
file_list = os.listdir(collection)
# for i in range(len(file_list)):
# for i in range(n_test):
#     print(file_list[i])
print(len(file_list))

2


In [4]:
collection = "./html"
# data_folder = Path(collection)
output_folder = PureWindowsPath('C:/Users/T.Santos18/Documents/workspace/textparsing/exec')
# input_folder = PureWindowsPath('C:/Users/T.Santos18/Documents/workspace/textparsing/html')

output_folder = Path('./output/')
input_folder = Path(collection)


n_test=1
file_list = os.listdir(collection)
out_list =[]
# for i in range(len(file_list)):
print('FILE\tANALYST_Flag\tFIRM\tQuarter\tYear') 
print(len(file_list))

FILE	ANALYST_Flag	FIRM	Quarter	Year
2


In [134]:
contents = open(str( / file_list[i]), "r").read()

SyntaxError: invalid syntax (<ipython-input-134-47003571f59b>, line 1)

In [5]:
out_list = []
for i in range(len(file_list)):
    if(re.search('html',file_list[i],flags=re.IGNORECASE)):
        out_list.append(file_list[i])
        print(file_list[i])


5717.html


In [6]:
out_file = pd.DataFrame(out_list,columns=['filename'])
out_file.index.names=['index']
out_file.to_csv('all_html.csv',encoding='utf-8')

In [7]:
out_file.iloc[0]['filename']
len(out_file)

1

In [8]:
output_folder = Path('./sample_out')
input_folder = Path(collection)
# print(input_folder)

In [9]:
display(output_folder)
with open(str( output_folder/'test_extract.csv'), "w+") as csvfile:
    print('f')
    logwriter = csv.writer(csvfile,delimiter='\t',quotechar='"',quoting=csv.QUOTE_MINIMAL)
    logwriter.writerow(['1','1','fail'])

PosixPath('sample_out')

f


In [208]:
ii = 0
test =[out_file.iloc[0].filename]
test
test = pd.DataFrame(test,columns=['filename'])
test
type(test)

pandas.core.frame.DataFrame

In [111]:
output_folder = Path('./sample_out')

with open(str(output_folder / 'test_extract.csv'), "w+") as csvfile:

    logwriter = csv.writer(csvfile,delimiter='\t',quotechar='"',quoting=csv.QUOTE_MINIMAL)
    i_start = 0
    i_end = 1
 
    for ii in range(i_start,i_end):
        try:
            #batch
            #basically 1 file at a time now
            row_start = ii*1
            row_end = (ii*1)+1
            filename = input_folder /  out_file.iloc[0]['filename']

            output_filename = 'out_' + str(ii) + '.csv'
            output_filename_exec = 'out_exec_' + str(ii) + '.csv'
            output_filename_analyst = 'out_analyst_' + str(ii) + '.csv'

#             for i in range(row_start,row_end):
#                 out_list.append(file_list[i])
            contents = extract_html(str(filename))
            print('here')
            out_pd = out_file
            out_pd['text'] = out_pd.apply(lambda x: extract_html(str(input_folder / x['filename'])), axis=1)
            out_pd['quarter'] = out_pd.apply(lambda x: extract_quarter(x['text']), axis=1)
            out_pd['call_date'] = out_pd.apply(lambda x: extract_time(x['text']), axis=1)
            out_pd['year'] = out_pd.apply(lambda x: extract_year(x['text']), axis=1)
            out_pd['executive_firm'] = out_pd.apply(lambda x: extract_firm(x['text']), axis=1)
            out_pd['ticker'] = out_pd.apply(lambda x: extract_ticker(x['text']), axis=1)
            out_pd['analysts'] = out_pd.apply(lambda x: extract_analysts(x['text']), axis=1)
            out_pd['execs'] = out_pd.apply(lambda x: extract_executives(x['text']), axis=1)
            
            s = out_pd.execs.apply(pd.Series)\
                          .stack()\
                          .reset_index(level=1, drop=True)\
                          .to_frame('executives')
            out_executives = out_pd.join(s)                      
            


            
            out_executives['exec_name'] = out_executives.apply(lambda x: extract_name(x['executives']), axis=1)
            out_executives['exec_title'] = out_executives.apply(lambda x: extract_title(x['executives']), axis=1)
#             out_pd = out_pd.drop(['text'],axis=1)

            out_executives['executive_content'] = out_executives.apply(lambda x: \
                                                     extract_content(contents,x['exec_name']), axis=1)
            out_executives['executive_qanda'] = out_executives.apply(lambda x: \
                                                     extract_qanda(contents,x['exec_name']), axis=1)
            out_executives=out_executives.reset_index(level=0, drop=True)
            
            s = out_executives.execs.apply(pd.Series)\
                          .stack()\
                          .reset_index(level=1, drop=True)\
                          .to_frame('mention')
            out_executives = out_executives.join(s)    
            
            out_executives['mention_name'] = out_executives.apply(lambda x: extract_name(x['mention']), axis=1)
            
            out_executives['mention_title'] = out_executives.apply(lambda x: extract_title(x['mention_name']), axis=1)
            
            out_executives['exec_mention_qna'] = out_executives.apply(lambda x: extract_name_pair(x['mention_name'],x['executive_qanda']), axis=1)
            
            out_executives['exec_mention_transcript'] = out_executives.apply(lambda x: extract_name_pair(x['mention_name'],x['executive_content']), axis=1)
     
        
        

            s = out_pd.analysts.apply(pd.Series)\
                          .stack()\
                          .reset_index(level=1, drop=True)\
                          .to_frame('analyst')
            out_analysts = out_pd.join(s)                 
            
            
            out_analysts['analyst_name'] = out_analysts.apply(lambda x: extract_name(x['analyst']), axis=1)
            out_analysts['analyst_firm'] = out_analysts.apply(lambda x: extract_title(x['analyst']), axis=1)

            out_analysts['analyst_content'] = out_analysts.apply(lambda x: \
                                                     extract_content(contents,x['analyst_name']), axis=1)
            
            out_analysts['analyst_qanda'] = out_analysts.apply(lambda x: \
                                                     extract_qanda(contents,x['analyst_name']), axis=1)

            out_analysts=out_analysts.reset_index(level=0, drop=True)

            s = out_analysts.execs.apply(pd.Series)\
                          .stack()\
                          .reset_index(level=1, drop=True)\
                          .to_frame('mention')
            out_analysts = out_analysts.join(s)                 
            out_analysts['mention_name'] = out_analysts.apply(lambda x: extract_name(x['mention']), axis=1)
            out_analysts['mention_title'] = out_analysts.apply(lambda x: extract_title(x['mention']), axis=1)
            
            out_analysts['exec_mention_qna'] = out_analysts.apply(lambda x: extract_name_pair(x['mention_name'],x['analyst_qanda']), axis=1)
            out_analysts['exec_mention_transcript'] = out_analysts.apply(lambda x: extract_name_pair(x['mention_name'],x['analyst_content']), axis=1)
            
            
#             out_executives['exec_qna_count'] = out_executives.apply(lambda x: \
#                                                      basic_wc(x['executive_qanda']), axis=1)

#             out_executives['exec_content_count'] = out_executives.apply(lambda x: \
#                                                      basic_wc(x['executive_content']), axis=1)
#             out_executives['exec_total_count'] = out_executives.exec_qna_count + out_executives.exec_content_count

#             out_pd=out_pd.reset_index(level=0, drop=True)
#             # out_analyst
#             s = out_pd.execs.apply(pd.Series)\
#                           .stack()\
#                           .reset_index(level=1, drop=True)\
#                           .to_frame('mention_name')
#             out_pd = out_pd.join(s)                      
                        
#             out_executives = out_executives.join(s)          
#             out_executives['mention_titile'] = out_executives.apply(lambda x: extract_title(x['mention_name']), axis=1)
#             out_executives['mention_name'] = out_executives.apply(lambda x: extract_name(x['mention_name']), axis=1)
#             out_executives['exec_mention_qna'] = out_executives.apply(lambda x: extract_name_pair(x['mention_name'],x['executive_qanda']), axis=1)
#             out_executives['exec_mention_transcript'] = out_executives.apply(lambda x: extract_name_pair(x['mention_name'],x['executive_content']), axis=1)
#             out_executives['mention_name_count_qna'] = out_executives.apply(lambda x: \
#                                           len(x['exec_mention_qna']), axis=1)
#             out_executives['mention_name_count_transcript'] = out_executives.apply(lambda x: \
#                                           len(x['exec_mention_transcript']), axis=1)
#             out_executives['mention_name_count_total']= out_executives.mention_name_count_qna+out_executives.mention_name_count_transcript
# #             print('here##')
#             # out_analyst.drop(['text','analyst_present','analysts','executive'],axis=1)
#             out_executives.drop(['text','executive','execs','exec_mention_qna',\
#                                  'exec_mention_transcript'],axis=1)\
#             .to_csv(output_folder / filename,sep=',', encoding = 'utf-8',header=False)
           
#         #             .to_csv(filename,sep=',', encoding = 'utf-8',header=False)
#             # out_executives.select({[}'file','quarter_yer'])    
# #             print('success: ', ii, ' ###')
#             logwriter.writerow([ii,out_executives.file.head(1).item(),'success'])
            out_pd \
    .to_csv(output_folder / output_filename,sep=',', encoding = 'utf-8',header=True)
            
#             out_executives \
#     .to_csv(output_folder / output_filename_exec,sep=',', encoding = 'utf-8',header=True)    
    
            out_executives \
    .drop(['text','executive_content','executive_qanda'],axis=1) \
    .to_csv(output_folder / output_filename_exec,sep=',', encoding = 'utf-8',header=True)    
            
#             out_analysts \
#     .to_csv(output_folder / output_filename_analyst,sep=',', encoding = 'utf-8',header=True)
    
            out_analysts \
    .drop(['text','analyst_content','analyst_qanda'],axis=1) \
    .to_csv(output_folder / output_filename_analyst,sep=',', encoding = 'utf-8',header=True)    
            print('here')
        except:
            print('exception: ', ii, ' ###')
#             logwriter.writerow([ii,out_executives.file.head(1).item(),'fail'])
            
            pass

here
here


In [55]:
def extract_time(input_soup):
    temp = input_soup.find(id='article_participants')
    for element in temp.find_all('p'):
        temp_string=element.text.strip()
        
        try:
#             December 2, 2014 11:15 AM ET
            
            temp_string = re.match(string=temp_string,pattern=\
                                   '^[^a-zA-Z]*([a-zA-Z]+\s+[0-9]+,\s+[0-9]{4})[^ 0-9]?(\s+[0-9]{1,2}:[0-9]{2}\s+[APap]\.?[mM]\.?).*')
            string1 = temp_string.group(1)
            string2 = temp_string.group(2)
            #         name_search.group(1).strip()
            string3 = string1 + string2
            string3 = re.sub(string=string3,pattern='\.',repl='')
            string3 = re.sub(string=string3,pattern='\s+',repl=' ')
#             print(string1,'##',string2,'##',string3)
            date_out = arrow.get(string3,'MMMM D, YYYY H:mm A')
        except:
            date_out=None        
            
        if date_out is not None:
            return date_out.isoformat()
    temp = input_soup.find(id='article_content')
    
    for element in temp.find_all('a'):
        temp_string=element.text.strip()
        temp_string = re.sub(string=temp_string,pattern='\s+',repl=' ')
        try:
            while  (element):
                try:
                    date_out = arrow.get(temp_string,'MMMM D, YYYY H:mm A')
                except:
                    date_out=None       
                if date_out is not None:
    #                 return temp_string
                    return date_out.isoformat()


                element = element.next_sibling
                temp_string = element
                temp_string = re.sub(string=temp_string,pattern='\s+',repl=' ')  
    #             print(temp_string.find('div'))
    #             print(temp_string.find('strong'))
        except:
            pass
    return None

In [106]:
def extract_quarter(input_soup):
    
    try:
        temp = input_soup.find(id='article_participants')
        for element in temp.findAll('p'):
            if (pd.notnull(element.text) and element.text!=''):
                if( re.search('q[1-4]\s*[0-9]{4}',element.contents[0],flags=re.IGNORECASE)):
                    print(element.contents[0])
                    temp_string = re.match(string=element.contents[0],\
                                           pattern='[^qQ]*([qQ][1-4])\s*[0-9]{4}.*')
                    print(temp_string.group(1))
                    call_title = element.contents[0].split()
                    return temp_string.group(1)
                
        temp = input_soup.find(id='article_source')        
        for element in temp.findAll('a'):
            if (pd.notnull(element.text) and element.text!=''):
                if( re.search('q[1-4]\s*[0-9]{4}',element.contents[0],flags=re.IGNORECASE)):
                    print(element.contents[0])
                    temp_string = re.match(string=element.contents[0],\
                                           pattern='[^qQ]*([qQ][1-4])\s*[0-9]{4}.*')
                    print(temp_string.group(1))
                    call_title = element.contents[0].split()
                    return temp_string.group(1)                
        return None
    except:

        return None
    
def extract_year(input_soup):
    
    try:
        temp = input_soup.find(id='article_participants')
        for element in temp.findAll('p'):
            if (pd.notnull(element.text) and element.text!=''):
                if( re.search('q[1-4]\s*[0-9]{4}',element.contents[0],flags=re.IGNORECASE)):
                    print(element.contents[0])
                    temp_string = re.match(string=element.contents[0],\
                                           pattern='[^qQ]*[qQ][1-4]\s*([0-9]{4}).*')
                    print(temp_string.group(1))
                    call_title = element.contents[0].split()
                    return temp_string.group(1)
                
        temp = input_soup.find(id='article_source')        
        for element in temp.findAll('a'):
            if (pd.notnull(element.text) and element.text!=''):
                if( re.search('q[1-4]\s*[0-9]{4}',element.contents[0],flags=re.IGNORECASE)):
                    print(element.contents[0])
                    temp_string = re.match(string=element.contents[0],\
                                           pattern='[^qQ]*[qQ][1-4]\s*([0-9]{4}).*')
                    print(temp_string.group(1))
                    call_title = element.contents[0].split()
                    return temp_string.group(1)                
        return None
    except:
        return None  

## Test Some Files

In [ ]:
import csv
fieldnames = ['index','fname', 'status']
# output_folder = PureWindowsPath('C:/Users/T.Santos18/Documents/workspace/textparsing/exec')
output_folder = PureWindowsPath('C:/Users/T.Santos18/Documents/workspace/textparsing/dates')
input_folder = PureWindowsPath('C:/Users/T.Santos18/Documents/workspace/textparsing/html')


with open(output_folder / 'log_extract.csv', 'w') as csvfile:

    logwriter = csv.writer(csvfile,delimiter='\t',quotechar='"',quoting=csv.QUOTE_MINIMAL)
    for ii in range(1,10):
        try:
            total_rows = len(file_list)
            row_start = ii*1
            row_end = (ii*1)+1
            filename = 'exec_out_foringestion_'+str(row_start)+'.csv'

            out_list =[]

            for i in range(row_start,row_end):
                with open(input_folder / file_list[i]) as inf:
#                 with open(collection + '/' + file_list[i]) as inf:
                    if(re.search('html',file_list[i],flags=re.IGNORECASE)):
                        out_list.append(file_list[i])

            out_pd = pd.DataFrame(out_list)
            out_pd.columns=['file']
            out_pd['text'] = out_pd.apply(lambda x: extract_html('html/'+x['file']), axis=1)
            out_pd['quarter'] = out_pd.apply(lambda x: extract_quarter(x['text']), axis=1)
            out_pd['year'] = out_pd.apply(lambda x: extract_year(x['text']), axis=1)
            out_pd['executive_firm'] = out_pd.apply(lambda x: extract_firm(x['text']), axis=1)
            out_pd['ticker'] = out_pd.apply(lambda x: extract_ticker(x['text']), axis=1)
            
            out_executives = out_pd                      
            out_executives.drop(['text'],axis=1).to_csv(output_folder / filename,sep=',', encoding = 'utf-8',header=False)
            logwriter.writerow([ii,out_executives.file.head(1).item(),'success'])
        except:
            print('exception: ', ii, ' ###')
            logwriter.writerow([ii,out_executives.file.head(1).item(),'fail'])
            
            pass